<a href="https://colab.research.google.com/github/cedricclg/Deep-Learning/blob/master/Taxi_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import json

from sklearn.cluster import MeanShift

import time
from datetime import datetime as dt

Runtime > Change runtime type > Hardware acceleration: GPU

In [0]:
device = torch.device("cuda:0")

In [0]:
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

Path to the data in Google Drive:

In [0]:
from google.colab import drive
drive.mount('/content/drive')

data_path = '/content/drive/My Drive/Deep-Learning/Deep-Learning/Data/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def get_data(file):
  df = pd.read_csv(data_path + file)
  return df

# **Validation and testing sets**

Shuffle the data, extract and save the new sets (CAN BE RUN ONLY ONCE, THE NEW DATA FILES ARE LOADED IN THE NEXT SECTIONS) \\
Validation: 20070 trajectories \\
Testing: 20000 trajectories

In [0]:
train = get_data("train.csv")

In [0]:
train = train.sample(frac=1).reset_index(drop=True)

In [0]:
valid = train.iloc[:20070].reset_index(drop=True)
test = train.iloc[20070:40070].reset_index(drop=True)
train = train.iloc[40070:].reset_index(drop=True)

In [0]:
train.to_csv(data_path + "train_new.csv")
valid.to_csv(data_path + "valid_new.csv")
test.to_csv(data_path + "test_new.csv")

# **Clustering**

Find and save the clusters for all the destinations of the new training set

In [0]:
train = get_data("train_new.csv")

Convert a list of strings into a list of coordinates:

In [0]:
def string_to_list(list_of_string):
  list_total = []
  for string in list_of_string:
    list_total.append(json.loads(string))
  return list_total

Extract the last coordinates of a string of successive coordinates:

In [0]:
def destination(string):
  l = len(string)
  if l!=2:
    for i in range(l):
      if string[l-i-1]=='[':
        new_string = string[l-i-1:l-1]
        break
  else:
    new_string = string
  dest = json.loads(new_string)
  dest = np.array(dest)
  return dest

All the destinations of the training set:

In [0]:
n = len(train['POLYLINE'])

destinations = np.zeros((n,2))
for i in range(n):
  if len(destination(train['POLYLINE'][i])) !=0: # Check if the trajectory (and the destination) actually exists
    destinations[i,:] = destination(train['POLYLINE'][i])
  else:
    destinations[i,:] = np.array([-8.611317,41.146504]) # If not, we manually choose the destination to be the main avenue

In [0]:
print(destinations)

[[-8.594073 41.161752]
 [-8.605467 41.126229]
 [-8.646741 41.148468]
 ...
 [-8.636535 41.206464]
 [-8.636994 41.172237]
 [-8.559027 41.231844]]


Number of samples (destinations) used for the clustering:

In [0]:
#n_samples_clusters = 300000

Run the clustering algorithm and save the results (CAN BE RUN ONLY ONCE, THE CLUSTERS ARE THEN LOADED IN THE NEXT SECTIONS)

Mean-Shift algorithm:

In [0]:
#mean_dest = np.mean(destinations,axis=0)
#std_dest = np.sqrt(np.var(destinations))

In [0]:
#centers = clustering.cluster_centers_

In [0]:
#C = centers.shape[0]
#print(C)
#print(centers)

In [0]:
#np.save(data_path+"centers", centers)

A (much) faster method for clustering, over all the training set: K-means algorithm

Install packages (run twice if it does not work):

In [0]:
!pip install sphinx > install.log
!pip install pykeops[full] > install.log

from pykeops.torch import LazyTensor

Desired number of clusters:

In [0]:
C = 3000

In [0]:
def KMeans(x, K=10, Niter=10, verbose=True):
    N, D = x.shape  # Number of samples, dimension of the ambient space

    # K-means loop:
    # - x  is the point cloud,
    # - cl is the vector of class labels
    # - c  is the cloud of cluster centroids
    start = time.time()
    c = x[:K, :].clone()  # Simplistic random initialization
    x_i = LazyTensor(x[:, None, :])  # (Npoints, 1, D)

    for i in range(Niter):

        c_j = LazyTensor(c[None, :, :])  # (1, Nclusters, D)
        D_ij = ((x_i - c_j) ** 2).sum(-1)  # (Npoints, Nclusters) symbolic matrix of squared distances
        cl = D_ij.argmin(dim=1).long().view(-1)  # Points -> Nearest cluster

        Ncl = torch.bincount(cl).type(torch.float64)  # Class weights
        for d in range(D):  # Compute the cluster centroids with torch.bincount:
            c[:, d] = torch.bincount(cl, weights=x[:, d]) / Ncl

    end = time.time()

    if verbose:
        print("K-means example with {:,} points in dimension {:,}, K = {:,}:".format(N, D, K))
        print('Timing for {} iterations: {:.5f}s = {} x {:.5f}s\n'.format(
                Niter, end - start, Niter, (end-start) / Niter))

    return cl, c

In [0]:
cl, centers = KMeans(torch.from_numpy(destinations).to(device),K=C)

Compiling libKeOpstorchd4132d0a01 in /root/.cache/pykeops-1.3-cpython-36/build-libKeOpstorchd4132d0a01:
       formula: ArgMin_Reduction(Sum(Square((Var(0,2,0) - Var(1,2,1)))),0)
       aliases: Var(0,2,0); Var(1,2,1); 
       dtype  : float64
... Done.
K-means example with 1,670,600 points in dimension 2, K = 3,000:
Timing for 10 iterations: 33.75463s = 10 x 3.37546s



In [0]:
print(centers)
print(centers.shape)

tensor([[-8.5941, 41.1617],
        [-8.6055, 41.1276],
        [-8.6460, 41.1442],
        ...,
        [-8.6542, 41.1806],
        [-8.6352, 41.1881],
        [-8.6012, 41.1816]], device='cuda:0', dtype=torch.float64)
torch.Size([3000, 2])


In [0]:
centers = centers.cpu().numpy()
centers = centers[~np.isnan(centers).any(axis=1),:]

In [0]:
print(centers)
C = centers.shape[0]
print(C)

[[-8.59411598 41.16172906]
 [-8.60547552 41.12759636]
 [-8.64602132 41.14424992]
 ...
 [-8.65419941 41.18062638]
 [-8.63515432 41.18811249]
 [-8.60124582 41.18163732]]
(2984, 2)


In [0]:
np.save(data_path+"centers_K", centers)

# **Define and train the model (MLP)**

Distances relevant to the problem:

In [0]:
R = 6371
pi = np.pi

# The distance are normalized such that R = 1

def equirectangular(x,y):
  dist = 1 * torch.sqrt( (y[1]-x[1]).pow(2) + ( (y[0]-x[0]) * torch.cos( 0.5*(y[1]-x[1]) ) ).pow(2) )
  return dist

def haversine(x,y):
  a = torch.sin( 0.5*(y[1]-x[1]) ).pow(2) + torch.cos(x[1])*torch.cos(y[1])*torch.sin( 0.5*(y[0]-x[0]) ).pow(2)
  dist = 2 * torch.atan( torch.sqrt( a/(1-a) ) )
  return dist

$x[0]$ is the tensor of the longitudes ($\lambda$, radians) \\
$x[1]$ is the tensor of the latitudes ($\phi$, radians)

Load the clusters from the mean-shift algorithm:

In [0]:
centers = np.load(data_path + "centers_K.npy")

In [0]:
C = centers.shape[0]
print(centers)
print(C)

[[-8.59411598 41.16172906]
 [-8.60547552 41.12759636]
 [-8.64602132 41.14424992]
 ...
 [-8.65419941 41.18062638]
 [-8.63515432 41.18811249]
 [-8.60124582 41.18163732]]
2984


Partial trajectories that will train our model are chosen to be the first 2k points:

In [0]:
k = 5

Define the model:

In [0]:
hidden_dim = 500

class Model(nn.Module):

  def __init__(self):
    super(Model,self).__init__()
    self.fc1 = nn.Linear(in_features=4*k , out_features=hidden_dim).to(device)
    self.relu = nn.ReLU().to(device)
    self.fc2 = nn.Linear(in_features=hidden_dim, out_features=C).to(device)
    self.centers = torch.from_numpy(centers).to(device)

  def forward(self,x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    x = F.softmax(x,dim=-1)
    cen = self.centers
    x = x.mm(cen)
    
    return x

In [0]:
model = Model()

Loss between two tensors: \\
$x$ predictions (batch_size,2), $y$ true destinations (batch_size,2) \\
Convert degrees to radians

In [0]:
def loss_fn(x,y):
  return equirectangular(torch.t(x*pi/180),torch.t(y*pi/180)).sum()

Training of the model:

In [0]:
nb_epochs = 30
batch_size = 200
lr = 0.01
momentum = 0.9

n_batches = n // batch_size

NameError: ignored

Convert a list of batch_size trajectories into an input array of partial trajectories of shape (batch_size , 2k , 2): \\
Only consider the first 2k points of each trajectory

In [0]:
def list_to_partial(X,batch_size=batch_size):
  
  Y = np.zeros((batch_size,2*k,2))
  
  for l in range(batch_size):
        
      if len(X[l]) == 0:                                # if the trajectory does not exist
        for j in range(k):                              
          Y[l,j,:] = np.array([-8.611317,41.146504])
          Y[l,2*k-1-j,:] = np.array([-8.611317,41.146504])

      elif len(X[l]) < k:                               # if the trajectory has less than k points
        diff = k - len(X[l])
        for j in range(diff+1):
          Y[l,j,:] = np.array( X[l][0] )
          Y[l,2*k-1-j,:] = np.array( X[l][len(X[l])-1] )
        for j in np.arange(diff+1,k,1):
          Y[l,j,:] = np.array( X[l][j-diff] )
          Y[l,2*k-1-j,:] = np.array( X[l][k-1-j] )  
        
      elif len(X[l]) < 2*k:                              # if the trajectory has more than k points but less than 2k
        for j in range(k):
          Y[l,j,:] = np.array( X[l][j] )
          Y[l,2*k-1-j,:] = np.array( X[l][len(X[l])-1-j] )

      else:                                             # if the trajectory has more than 2k points
        for j in range(k):
          Y[l,j,:] = np.array( X[l][j] )
          Y[l,2*k-1-j,:] = np.array( X[l][2*k-1-j] )

  return Y

Compute the mean and variance (or standard deviation) of the GPS points of all training trajectories with 2k prefixes:

In [0]:
mean_v = np.zeros((n_batches,2))
var_v = np.zeros((n_batches,2))

for i in range(n_batches):
  X = train['POLYLINE'][i*batch_size:(i+1)*batch_size]
  X = string_to_list(X)
  X = list_to_partial(X)
  mean_v[i] = np.mean(X,axis=(0,1))
  var_v[i] = np.var(X,axis=(0,1))

mean = np.mean(mean_v,axis=0)
std = np.sqrt(np.mean(var_v,axis=0))
print(mean,std)

[-8.61739435 41.15714826] [0.06085632 0.0222809 ]


Training loop:

In [0]:
model.train()
optimizer = torch.optim.SGD(model.parameters(),lr=lr,momentum=momentum)

In [0]:
model.double()
for epoch in range(nb_epochs):
  print(epoch)
  for i in range(n_batches):
    
    X = train['POLYLINE'][i*batch_size:(i+1)*batch_size]
    X = string_to_list(X)
    X = list_to_partial(X)

    # Standardized input GPS points
    X = ( X - mean ) / std
    
    X = np.reshape(X,(batch_size,4*k))
    X = torch.from_numpy(X).to(device)
    
    dest_pred = model(X.double())

    dest_true = destinations[i*batch_size:(i+1)*batch_size]
    dest_true = torch.from_numpy(dest_true).to(device)

    loss = loss_fn(dest_pred,dest_true)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [0]:
torch.save(model,data_path+"model.dat")

# **Test the model**

In [0]:
model = torch.load(data_path + "model.dat")

model.eval()

test = get_data("test_new.csv")

Get the true destinations of the trajectories in our test set:

In [0]:
n_test = len(test['POLYLINE'])

destinations_test = np.zeros((n_test,2))
for i in range(n_test):
  if len(destination(test['POLYLINE'][i])) !=0: # Check if the trajectory (and the destination) actually exists
    destinations_test[i,:] = destination(test['POLYLINE'][i])
  else:
    destinations_test[i,:] = np.array([-8.611317,41.146504]) # If not, we manually choose the destination to be the main avenue

partial_test = list_to_partial( string_to_list( test['POLYLINE'] ), batch_size=n_test )

# Remove absurd points:

if destinations_test.max(axis=0)[0]>-8.4:
  arg = np.argmax(destinations_test,axis=0)[0]
  destinations_test = np.delete(destinations_test,arg,axis=0)
  partial_test = np.delete(partial_test,arg,axis=0)
elif destinations_test.max(axis=0)[1]>41.3:
  arg = np.argmax(destinations_test,axis=0)[1]
  destinations_test = np.delete(destinations_test,arg,axis=0)
  partial_test = np.delete(partial_test,arg,axis=0)
elif destinations_test.min(axis=0)[0]<-8.8:
  arg = np.argmin(destinations_test,axis=0)[0]
  destinations_test = np.delete(destinations_test,arg,axis=0)
  partial_test = np.delete(partial_test,arg,axis=0)
elif destinations_test.min(axis=0)[1]<41.:
  arg = np.argmin(destinations_test,axis=0)[1]
  destinations_test = np.delete(destinations_test,arg,axis=0)
  partial_test = np.delete(partial_test,arg,axis=0)

n_test = destinations_test.shape[0]

destinations_test = torch.from_numpy(destinations_test).to(device)

Get the corresponding predicted destinations by our model:

In [0]:
partial_test = (partial_test - mean) / std

partial_test = np.reshape(partial_test,(n_test,4*k))
partial_test = torch.from_numpy(partial_test).to(device)
    
destinations_pred = model(partial_test.double())

Mean Haversine distance between the predictions and reality:

In [0]:
accuracy = R * haversine(torch.t(destinations_pred*pi/180),torch.t(destinations_test*pi/180)).mean().item()

Accuracy in kilometers:

In [0]:
print(accuracy.item())

2.4661206369800635


Accuracy in degres:

In [0]:
print((destinations_pred-destinations_test).mean(axis=0))

tensor([ 0.0014, -0.0033], device='cuda:0', dtype=torch.float64,
       grad_fn=<MeanBackward1>)


# **Another model: with metadata and all possible prefixes for the training set**

Before explaining the feature engineering, it is important to explain how the program will work. Since we are dealing with 1.7 millions of data in which each data has a complete trajectory (we have to divide all of them in all possible partial trajectory), the RAM memory is not enough to store such amount. Then, we will divide de dataset in several parts (batch) and for each part well determined, we will clean/organize them and train the model.

The class **OurFeatureEngineering** will be charged of dividing the dataset, cleaning and organizing each batch, and providing the organized batch (*next_batch*) until there is no more data.

In [0]:
train = get_data("train_new.csv")

In [0]:
batch_size = 200
k = 5

In [0]:
class OurFeatureEngineering:

  #### ------------------------------
  #### Initializing the class
  #### ------------------------------
  # INPUT
  # df_train (pd.DataFrame) -> The full dataset, but we store without the trajectories
  # batch_size (int) -> The size of a batch
  # k (int) -> The quantity of step at the beginning and the end for the construction of a partial trajectory
  # VARIABLES
  # list_string (list) -> it stores the string of all complete trajectories
  # n_it (int) -> it indicates the next data of the dataset we have to use for the next batch
  # length (int) -> it stores the quantity of data in the stored dataset
  def __init__(self, df_train, batch_size=batch_size, k=k):
    # self.df_train = df_train.loc[:, df_train.columns != 'POLYLINE'].reset_index(drop=True)
    # self.list_string = list(df_train["POLYLINE"])
    self.df_train, self.list_string = self.df_filter(df_train)
    self.batch_size = batch_size
    self.k = k
    
    self.n_it = 0
    self.length = self.df_train.shape[0]


  #### ------------------------------
  #### Filtering the lines and columns of the dataset
  #### ------------------------------
  def df_filter(self, df):
    # Drop the null trajectories
    df = df[df["POLYLINE"] != "[]"]

    # Drop the incomplete trajectories and drop off the MISSING_DATA column
    df = df[df["MISSING_DATA"] == False]
    df = df.drop(columns=["MISSING_DATA"])

    # Change the CALL_TYPE values
    df = df.replace({"CALL_TYPE": {"A":1,"B":2, "C":3}})

    # ORIGIN_CALL -> CLIENT_ID and ORIGIN_STAND -> STAND_ID
    df = df.rename(columns={"ORIGIN_CALL": "CLIENT_ID", "ORIGIN_STAND": "STAND_ID"})

    # Just the CLIENT_ID and STAND_ID can have null values. Since they are
    # always greater than 0, we will do null_values -> -1
    df = df.fillna(-1)

    # Drop off the DAY_TYPE column because all the values are A
    df = df.drop(columns=["DAY_TYPE"])

    # Changing the values of ORIGIN_CALL, ORIGIN_STAND and TAXI_ID
    change_client = {k : v for v,k in enumerate(sorted(df["CLIENT_ID"].unique())) }
    change_stand = {k : v for v,k in enumerate(sorted(df["STAND_ID"].unique())) }
    change_taxi = {k : v for v,k in enumerate(sorted(df["TAXI_ID"].unique())) }
    df["CLIENT_ID"] = df["CLIENT_ID"].map(change_client)
    df["STAND_ID"] = df["STAND_ID"].map(change_stand)
    df["TAXI_ID"] = df["TAXI_ID"].map(change_taxi)

    return df.drop(columns=["POLYLINE"]).reset_index(drop=True), list(df["POLYLINE"])
  
  #### ------------------------------
  #### Providing the next batch of data
  #### ------------------------------
  # It analyses if it is still possible to provide a batch. If yes, it returns it cleaned/organized
  # OUTPUT
  # The cleaned/organized batch (pd.DataFrame) 
  def next_batch(self):
    if self.n_it == self.length:
      return None

    # Any complete batch
    if self.n_it + self.batch_size < self.length:
      df = self.df_train.iloc[self.n_it : self.n_it+self.batch_size]
      self.n_it += self.batch_size
    # The last batch (which can have less elements)
    else:
      df = self.df_train.iloc[self.n_it : self.length]
      self.n_it = self.length
    
    # We organize/clean the batch and return it
    return self.preprocessing(df)
  

  #### ------------------------------
  #### Preprocessing the batch dataset
  #### ------------------------------
  # It calls all the steps to clean/organize the batch dataset
  # INPUT
  # df (pd.DataFrame) -> a batch dataset
  # INTERMEDIATE VARIABLES
  # ls (list) -> list of all prefixes of the batch dataset (it is not a list of string 
  #              like self.list_string, but a list of list of floats)
  # OUTPUT
  # The batch dataset organized/cleaned (Pd.DataFrame)
  def preprocessing(self, df):
    df = self.destinations(df)
    df, ls = self.all_prefixes(df)
    df = self.type_organise(df)
    df = self.creation_features(df, ls)
    df = self.taking_2k_coordinates(df, ls)
    return df.drop(columns=["TIMESTAMP", "TIMESTAMP_END", "TRIP_ID"])

  #### ------------------------------
  #### Adding the destination in the batch dataset
  #### ------------------------------
  # INPUT
  # df (pd.DataFrame) -> The batch dataset
  # OUTPUT
  # the batch dataset with the final destinations (pd.DataFrame)
  def destinations(self, df):
    dest = []
    list_string = [self.list_string[df.index.values.tolist()[i]] for i in range(df.shape[0])]

    # For each trajectory in string, we take the destination
    for i, string in enumerate(list_string):
      list_aux = json.loads(string)
      if len(list_aux) > 0:
        dest.append(list_aux[-1])
      else:
        dest.append([0,0])

    df_aux = pd.DataFrame(np.array(dest), columns=["dest_lon", "dest_lat"], dtype=np.float64)
    df_aux = df_aux.set_index(df.index)
    return pd.concat([df, df_aux], axis=1, sort=False)


  #### ------------------------------
  #### Organizing the type of the features of the batch dataset
  #### ------------------------------
  def type_organise(self, df):     
    df["TRIP_ID"] = df["TRIP_ID"].astype('object')
    df["CALL_TYPE"] = df["CALL_TYPE"].astype('int64')
    df["CLIENT_ID"] = df["CLIENT_ID"].fillna(-1).astype('int64')
    df["STAND_ID"] = df["STAND_ID"].fillna(-1).astype('int64')
    df["TAXI_ID"] = df["TAXI_ID"].fillna(-1).astype('int64')
    df["TIMESTAMP"] = df["TIMESTAMP"].fillna(-1).astype('int64')
    df["dest_lon"] = df["dest_lon"].astype('float64')
    df["dest_lat"] = df["dest_lat"].astype('float64')
    return df


  #### ------------------------------
  #### Creating all possible prefixes (partial trajectories) of the complete trajectories of the batch dataset
  #### ------------------------------
  # INPUT
  # df (pd.DataFrame) -> The batch dataset
  # OUTPUT
  # df_new (pd.DataFrame) -> It replicates the input batch dataset according to the number of prefixes.
  # list_total (list) -> It is a list that stores the prefixes (it is no more string like self.list_string, 
  #                      it is a list of floats) with respect to the df_new. 
  def all_prefixes(self, df):
    df_new = pd.DataFrame(columns=df.columns)
    length_total = []
    list_total = []
    list_string = [self.list_string[df.index.values.tolist()[i]] for i in range(df.shape[0])]

    # For each trajectory, we create all the prefixes
    for i, string in enumerate(list_string):
      list_aux = json.loads(string)
      length_total.append(len(list_aux))
      for j in range(1, len(list_aux) + 1):
        list_total.append(list_aux[:j])
    
    # Replicate the lines in the dataset
    df_new = pd.DataFrame(np.repeat(df.values,length_total,axis=0))
    df_new.columns = df.columns

    return df_new, list_total
  

  #### ------------------------------
  #### Creating other features
  #### ------------------------------
  # It increases the features of the batch dataset
  def creation_features(self, df, ls):
    df['LEN'] = [len(ls[i]) for i in df.index.values.tolist()]
    df['TIMESTAMP_END'] = df['TIMESTAMP'].map(int) + df['LEN'].map(lambda x: 15*x)
    df["WEEK"] = df['TIMESTAMP_END'].map(lambda x: dt.utcfromtimestamp(x).isocalendar()[1]) - 1
    df["DAY_WEEK"] = df['TIMESTAMP_END'].map(lambda x: dt.utcfromtimestamp(x).isocalendar()[2]) - 1
    df['QUARTER_HOUR'] = df['TIMESTAMP_END'].map(lambda x: 4*dt.utcfromtimestamp(x).hour + dt.utcfromtimestamp(x).minute // 15)
    return df
  

  #### ------------------------------
  #### Adding to the batch dataset the k firsts and k lasts coordinates of the prefixes
  #### ------------------------------
  # INPUT
  # df (pd.DataFrame) -> the batch dataset with the replicated lines
  # ls (list) -> The list of the prefixes with respect to df.
  # OUTPUT
  # The initial dataset with the 2k coordinates as features (pd.DataFrame)
  def taking_2k_coordinates(self, df, ls):
    k = self.k

    # Naming the columns
    colonnes = []
    for i in range(k):
      colonnes += ["lon_"+str(i+1), "lat_"+str(i+1)]
    for i in range(k, 0, -1):
      colonnes += ["lon_@"+str(i), "lat_@"+str(i)]

    df_aux = pd.DataFrame(columns=colonnes)
    
    # For each prefixes, we take the 2k points
    list_rows = []
    for i, list_aux in enumerate(ls):
      row = []
      n = len(list_aux)
      
      if n >= k:
        for j in range(k):
          row += list_aux[j]
        for j in range(k):
          row += list_aux[-k+j]
      elif n>0:
        for j in range(k-n):
          row += list_aux[0]
        for j in range(n):
          row += list_aux[j]
        for j in range(n):
          row += list_aux[-n+j]
        for j in range(k-n):
          row += list_aux[-1]
      else:
        continue
      list_rows.append(row)

    df_aux = pd.DataFrame(np.array(list_rows), columns=colonnes)
    df_aux = df_aux.set_index(df.index)
    return pd.concat([df, df_aux], axis=1, sort=False)

  #### ------------------------------
  #### Boolean that says if there are more batch
  #### ------------------------------
  def has_next(self):
    if self.n_it == self.length:
      return False
    return True

In [0]:
def reshaping(df, k=k):
  coord_col = []
  for i in range(k):
    coord_col += ["lon_"+str(i+1), "lat_"+str(i+1)]
  for i in range(k, 0, -1):
    coord_col += ["lon_@"+str(i), "lat_@"+str(i)]

  dest_col = ["dest_lon","dest_lat"]

  metadata = [col for col in df.columns if col not in coord_col + dest_col]

  return df[metadata], np.array(df[coord_col]).reshape(-1,2*k,2), np.array(df[dest_col])

In [0]:
features = OurFeatureEngineering(train)
del(train)
print(features.df_train)

         Unnamed: 0              TRIP_ID  ...  TAXI_ID   TIMESTAMP
0                 0  1385025300620000107  ...       77  1385025300
1                 1  1398009813620000686  ...      426  1398009813
2                 2  1400814708620000648  ...      401  1400814708
3                 3  1373778035620000600  ...      373  1373778035
4                 4  1380261303620000545  ...      343  1380261303
...             ...                  ...  ...      ...         ...
1664833     1670595  1386552971620000080  ...       60  1386552971
1664834     1670596  1384152481620000671  ...      415  1384152481
1664835     1670597  1377950684620000142  ...       98  1377950684
1664836     1670598  1372898324620000013  ...       12  1372898324
1664837     1670599  1388332298620000006  ...        5  1388332298

[1664838 rows x 7 columns]


Distances:

In [0]:
R = 6371
pi = np.pi

# The distance are normalized such that R = 1

def equirectangular(x,y):
  dist = 1 * torch.sqrt( (y[1]-x[1]).pow(2) + ( (y[0]-x[0]) * torch.cos( 0.5*(y[1]-x[1]) ) ).pow(2) )
  return dist

def haversine(x,y):
  a = torch.sin( 0.5*(y[1]-x[1]) ).pow(2) + torch.cos(x[1])*torch.cos(y[1])*torch.sin( 0.5*(y[0]-x[0]) ).pow(2)
  dist = 2 * torch.atan( torch.sqrt( a/(1-a) ) )
  return dist

Compute and savethe mean and variance (or standard deviation) of the GPS points of all training trajectories with 2k prefixes: \\
(CAN BE RUN ONLY ONCE)

In [0]:
mean_v = []
var_v = []

while features.has_next():
  _,X,_ = reshaping(features.next_batch())
  mean_v.append(np.mean(X,axis=(0,1)))
  var_v.append(np.var(X,axis=(0,1)))

mean_v = np.array(mean_v)
var_v = np.array(var_v)
mean = np.mean(mean_v,axis=0)
std = np.sqrt(np.mean(var_v,axis=0))
print(mean,std)
features.n_it = 0

In [0]:
np.save(data_path + 'mean',mean)
np.save(data_path + 'std',std)

Create the embeddings:

In [0]:
embed_dim = 10
total_client_id = 57106
total_taxi_id = 448
total_stand_id = 64
total_quarter = 96
total_day = 7
total_week = 52

In [0]:
embed_client_id = nn.Embedding(total_client_id,embed_dim)
embed_taxi_id = nn.Embedding(total_taxi_id,embed_dim)
embed_stand_id = nn.Embedding(total_stand_id,embed_dim)
embed_quarter = nn.Embedding(total_quarter,embed_dim)
embed_day = nn.Embedding(total_day,embed_dim)
embed_week = nn.Embedding(total_week,embed_dim)

Load the clusters:

In [0]:
centers = np.load(data_path + "centers_K.npy")

In [0]:
C = centers.shape[0]
print(centers)
print(C)

[[-8.59411598 41.16172906]
 [-8.60547552 41.12759636]
 [-8.64602132 41.14424992]
 ...
 [-8.65419941 41.18062638]
 [-8.63515432 41.18811249]
 [-8.60124582 41.18163732]]
2984


New model, with metadata (the input layer has now a shape (batch_size , 4k+60)):

In [0]:
hidden_dim = 500

class Model_2(nn.Module):

  def __init__(self):
    super(Model_2,self).__init__()
    self.fc1 = nn.Linear(in_features=4*k+6*embed_dim , out_features=hidden_dim).to(device)
    self.relu = nn.ReLU().to(device)
    self.fc2 = nn.Linear(in_features=hidden_dim, out_features=C).to(device)
    self.centers = torch.from_numpy(centers).to(device)

  def forward(self,x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    x = F.softmax(x,dim=-1)
    cen = self.centers
    x = x.mm(cen)
    
    return x

In [0]:
model2 = Model_2()

In [0]:
model2 = torch.load(data_path+"model2_new.dat")

Loss:

In [0]:
def loss_fn(x,y):
  return equirectangular(torch.t(x*pi/180),torch.t(y*pi/180)).sum()

Training:

In [0]:
mean = np.load(data_path + 'mean.npy')
std = np.load(data_path + 'std.npy')
print(mean,std)

[-8.61687434 41.15820505] [0.04636961 0.05601787]


In [0]:
lr = 0.01
momentum = 0.9

In [0]:
model2.train()
optimizer = torch.optim.SGD(model2.parameters(),lr=lr,momentum=momentum)

In [0]:
nb_epochs = 1

model2.double()
for epoch in range(nb_epochs):
  features.n_it = 0
  print(epoch)
  while features.has_next():
    
    X = features.next_batch()
    metadata, X, dest_true = reshaping(X)

    # Standardized input GPS points
    X = ( X - mean ) / std

    X = np.reshape(X,(X.shape[0],4*k))
    X = torch.from_numpy(X).to(device)
    
    meta_client_id = torch.from_numpy(np.array(metadata['CLIENT_ID']))
    meta_client_id = embed_client_id(meta_client_id)
    meta_client_id = meta_client_id.to(device).double()

    meta_taxi_id = torch.from_numpy(np.array(metadata['TAXI_ID']))
    meta_taxi_id = embed_taxi_id(meta_taxi_id)
    meta_taxi_id = meta_taxi_id.to(device).double()

    meta_stand_id = torch.from_numpy(np.array(metadata['STAND_ID']))
    meta_stand_id = embed_stand_id(meta_stand_id)
    meta_stand_id = meta_stand_id.to(device).double()

    meta_quarter = torch.from_numpy(np.array(metadata['QUARTER_HOUR']))
    meta_quarter = embed_quarter(meta_quarter)
    meta_quarter = meta_quarter.to(device).double()

    meta_day = torch.from_numpy(np.array(metadata['DAY_WEEK']))
    meta_day = embed_day(meta_day)
    meta_day = meta_day.to(device).double()

    meta_week = torch.from_numpy(np.array(metadata['WEEK']))
    meta_week = embed_week(meta_week)
    meta_week = meta_week.to(device).double()

    X = torch.cat((X,meta_client_id,meta_taxi_id,meta_stand_id,meta_quarter,meta_day,meta_week),1)

    dest_pred = model2(X.double())

    dest_true = torch.from_numpy(dest_true).to(device)

    loss = loss_fn(dest_pred,dest_true)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  torch.save(model2,data_path+"model_2.dat")

0


In [0]:
#torch.save(model2,data_path+"model_2.dat")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Model_2. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


# **Test the model:**

In [0]:
model2 = torch.load(data_path+"model2_new.dat")
model2.eval()

Model_2(
  (fc1): Linear(in_features=80, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=2984, bias=True)
)

In [0]:
test = get_data("test_new.csv")
features_test = OurFeatureEngineering(test.iloc[:500],batch_size=200)

Get the predictions by our model and the true destinations for the testing dataset:

In [0]:
features_test.n_it = 0

# Count the total number of trajectories in the extended testing dataset (with all possible prefixes)
count = 0

accuracy = 0.

while features_test.has_next():

  X = features_test.next_batch()
  metadata, X, dest_true = reshaping(X)

  # Remove absurd points:

  if dest_true.max(axis=0)[0]>-8.4:
    arg = np.argmax(dest_true,axis=0)[0]
    dest_true = np.delete(dest_true,arg,axis=0)
    X = np.delete(X,arg,axis=0)
    metadata = metadata.drop(arg)
  elif dest_true.max(axis=0)[1]>41.3:
    arg = np.argmax(dest_true,axis=0)[1]
    dest_true = np.delete(dest_true,arg,axis=0)
    X = np.delete(X,arg,axis=0)
    metadata = metadata.drop(arg)
  elif dest_true.min(axis=0)[0]<-8.8:
    arg = np.argmax(dest_true,axis=0)[0]
    dest_true = np.delete(dest_true,arg,axis=0)
    X = np.delete(X,arg,axis=0)
    metadata = metadata.drop(arg)
  elif dest_true.max(axis=0)[0]<41.:
    arg = np.argmax(dest_true,axis=0)[1]
    dest_true = np.delete(dest_true,arg,axis=0)
    X = np.delete(X,arg,axis=0)
    metadata = metadata.drop(arg)

  count += X.shape[0]

  # Standardized input GPS points
  X = ( X - mean ) / std

  X = np.reshape(X,(X.shape[0],4*k))
  X = torch.from_numpy(X).to(device)
    
  meta_client_id = torch.from_numpy(np.array(metadata['CLIENT_ID']))
  meta_client_id = embed_client_id(meta_client_id)
  meta_client_id = meta_client_id.to(device).double()

  meta_taxi_id = torch.from_numpy(np.array(metadata['TAXI_ID']))
  meta_taxi_id = embed_taxi_id(meta_taxi_id)
  meta_taxi_id = meta_taxi_id.to(device).double()

  meta_stand_id = torch.from_numpy(np.array(metadata['STAND_ID']))
  meta_stand_id = embed_stand_id(meta_stand_id)
  meta_stand_id = meta_stand_id.to(device).double()

  meta_quarter = torch.from_numpy(np.array(metadata['QUARTER_HOUR']))
  meta_quarter = embed_quarter(meta_quarter)
  meta_quarter = meta_quarter.to(device).double()

  meta_day = torch.from_numpy(np.array(metadata['DAY_WEEK']))
  meta_day = embed_day(meta_day)
  meta_day = meta_day.to(device).double()

  meta_week = torch.from_numpy(np.array(metadata['WEEK']))
  meta_week = embed_week(meta_week)
  meta_week = meta_week.to(device).double()

  X = torch.cat((X,meta_client_id,meta_taxi_id,meta_stand_id,meta_quarter,meta_day,meta_week),1)

  dest_pred = model2(X.double())

  dest_true = torch.from_numpy(dest_true).to(device)

  accuracy = accuracy + haversine(torch.t(dest_true*pi/180),torch.t(dest_pred*pi/180)).sum().item()
  
accuracy = accuracy / count
print(count)

23306


Accuracy in kilometers:

In [0]:
print(accuracy*R)

4.1488310106303405


Test the first basic model with the same testing set:

In [0]:
model = torch.load(data_path+"model.dat")
model.eval()

Model(
  (fc1): Linear(in_features=20, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=2984, bias=True)
)

In [0]:
test = get_data("test_new.csv")
features_test = OurFeatureEngineering(test.iloc[:1000],batch_size=1000)

In [0]:
features_test.n_it = 0

X = features_test.next_batch()
metadata, X, dest_true = reshaping(X)
X = np.reshape(X,(X.shape[0],4*k))
X = torch.from_numpy(X).to(device)
dest_pred = model(X.double())
dest_true = torch.from_numpy(dest_true).to(device)

accuracy = R*haversine(torch.t(dest_true*pi/180),torch.t(dest_pred*pi/180)).mean().item()

print(accuracy)

5.729494895531275
